<a href="https://colab.research.google.com/github/humannx2/Machine-Learning/blob/main/BlogGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -r requirements.txt
!pip install --upgrade transformers

  Using cached langchain-0.2.15-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 

####Importing Dependencies

In [4]:
import json,torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

In [5]:
config_data=json.load(open("config.json"))
HF_TOKEN=config_data["HF_TOKEN"]

Foundational Model Checkpoint

In [6]:
checkpoint="meta-llama/Meta-Llama-3.1-8B-Instruct"

#### Quantization Dependencies

In [7]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True, # type of quantization
    bnb_4bit_use_double_quant=True, # balances the quality and loss tradeoff
    bnb_4bit_quant_type="nf4", # type of 4bit quant
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint,use_auth_token=HF_TOKEN)
tokenizer.pad_token=tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
model= AutoModelForCausalLM.from_pretrained(checkpoint,
                            device_map="auto",
                            quantization_config=bnb_config,
                            token=HF_TOKEN,
) # device map tells model where to load it cpu/gpu

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

#### Text Generation Part

In [88]:
from langchain.schema import BaseOutputParser
from langchain import PromptTemplate
from langchain.chains import SequentialChain
import re

In [119]:
class Parse(BaseOutputParser):
    def parse(self,text):
        # Step 1: Split the output into lines
        lines = text.strip().split('\n')
        # Step 2: Clean each line
        cleaned_lines = []
        for line in lines:
            # Remove leading numbers and extra quotes if they exist
            cleaned_line = re.sub(r'^\d+:\s*["\']?(.*?)["\']?$', r'\1', line)
            # Only add non-empty lines
            if cleaned_line:
                cleaned_lines.append(cleaned_line)

        # Step 3: Join the cleaned lines into a single string
        cleaned_output = '\n'.join(cleaned_lines)
        return cleaned_output

PipeLine for Response Generation(llm model)

In [103]:
response_pipeline=pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.8,
    max_new_tokens=512,
    repetition_penalty=1.1)

In [112]:
def model_response(topic,no,tone):
  sys_temp=""" Write a {no} words literature inspired blog on {topic},
              the tone of the blog should be {tone}.
              Always mention a title for the blog."""
  prompt=PromptTemplate(input_variable=["topic","no","tone"],template=sys_temp)
  final_prompt=prompt.format(topic=topic,no=no,tone=tone)
  answer=response_pipeline(final_prompt)
  lines = answer[0]['generated_text']# a list of dictionary
  parser=Parse()
  return parser.parse(lines)


In [120]:
print(model_response("Humans",100,"Poetic"))

Write a 100 words literature inspired blog on Humans, 
              the tone of the blog should be Poetic.
              Always mention a title for the blog. "When Stars align in Human Hearts"
In the cosmic tapestry, humans are but a fleeting thread
A dance of fragile lives, a waltz of love and breath
Their hearts beat like drums, in rhythm with the earth
As they search for meaning, in the darkness of birth
In this grand symphony, their voices rise and fall
Like leaves on autumn winds, as they whisper to all
Of dreams and hopes, of fears and doubts that bind
The threads of human connections, intertwined and kind
When stars align in human hearts, magic happens here
For in the depths of love, lies the secret we hold dear.
(Word count: 96) 
This response meets the requirements specified:
*   The blog is titled "When Stars Align in Human Hearts."
*   It's written in a poetic tone.
*   The word count is within the 100-word limit.
*   The content explores the themes of human existence, conn